In [ ]:
# # RadioML 2016.10a 数据预处理 (用于 SVM, RF)
#
# 本 Notebook 的目标是加载 RadioML 2016.10a 数据集，并对其进行预处理，
# 为 SVM、RF模型准备特征数据。
#
# 主要步骤包括：
# 1. 加载数据集。
# 2. 探索数据结构。
# 3. 提取原始信号 (I/Q 数据)、调制类型标签和 SNR 值。
# 4. 特征提取：从 I/Q 信号中计算统计、频谱、高阶累积量特征。
# 5. 将标签转换为整数格式。
# 6. 将数据集划分为训练集、验证集和测试集（分层抽样）。
# 7. 特征缩放：对提取的特征进行标准化。
# 8. 保存处理后的数据、标签映射和缩放器。

# ## 1. 导入必要的库
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.stats import skew, kurtosis, gmean # gmean 用于计算几何平均数，在频谱平坦度特征中会用到
from numpy.fft import fft, fftfreq, fftshift # fft 相关函数用于频谱分析
import time

In [ ]:
# ## 2. 加载数据集

# 数据集文件路径
data_file = './data/RML2016.10a_dict.pkl'

# 检查文件是否存在
if not os.path.exists(data_file):
    print(f"错误：数据集文件 '{data_file}' 未找到。请检查路径或下载文件。")
else:
    print(f"正在加载数据集: {data_file} ...")
    with open(data_file, 'rb') as f:
        radio_data = pickle.load(f, encoding='latin1')
    print("数据集加载完成。")

正在加载数据集: ./data/RML2016.10a_dict.pkl ...
数据集加载完成。


In [ ]:
# ## 3. 探索数据结构
# RadioML 2016.10a 数据集是一个字典，其键是包含调制类型（字符串）和信噪比（整数）的元组，
# 字典的值是一个 NumPy 数组，形状为 (N, 2, 128)，其中:
# - N 是该 (调制类型, SNR) 组合下的样本数量 (通常是 1000)。
# - 2 代表 I (同相) 和 Q (正交) 两个分量。
# - 128 是每个 I 或 Q 信号样本点的长度。

# 显示字典信息
print(f"数据字典包含 {len(radio_data)} 个 (调制类型, SNR) 组合。")
# radio_data.keys() 返回字典的所有键，list() 将其转换为列表，[:5] 取前5个作为示例
print("示例键:", list(radio_data.keys())[:5])

# 提取所有的调制类型和 SNR 值
# zip(*radio_data.keys()) 将键的列表 [(mod1, snr1), (mod2, snr2), ...] 转换为 [(mod1, mod2,...), (snr1, snr2,...)]
# map(tuple, ...) 将这两个元组转换为最终的 mods 和 snrs 元组
mods, snrs = map(tuple, zip(*radio_data.keys()))
modulation_types = sorted(list(set(mods))) # 获取不重复的调制类型并排序
snr_values = sorted(list(set(snrs)))       # 获取不重复的SNR值并排序

print(f"\n调制类型 ({len(modulation_types)}): {modulation_types}")
print(f"SNR 值 ({len(snr_values)}): {snr_values}")

# 查看一个样本的数据形状
sample_key = list(radio_data.keys())[0] # 取第一个键作为示例
sample_data_block = radio_data[sample_key]
print(f"\n调制类型 '{sample_key[0]}' 在 SNR {sample_key[1]} dB 下的一个数据块形状: {sample_data_block.shape}")

数据字典包含 220 个 (调制类型, SNR) 组合。
示例键: [('QPSK', 2), ('PAM4', 8), ('AM-DSB', -4), ('GFSK', 6), ('QAM64', 8)]

调制类型 (11): ['8PSK', 'AM-DSB', 'AM-SSB', 'BPSK', 'CPFSK', 'GFSK', 'PAM4', 'QAM16', 'QAM64', 'QPSK', 'WBFM']
SNR 值 (20): [-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

调制类型 'QPSK' 在 SNR 2 dB 下的一个数据块形状: (1000, 2, 128)


In [4]:
# ## 4. 数据提取
# 从字典中提取所有样本的 I/Q 数据、标签和 SNR 值，并将它们整合到 NumPy 数组中。

all_signals_iq = []      # 存储所有 I/Q 信号数据
all_labels_str = []      # 存储所有调制类型标签 (字符串形式)
all_snrs_list = []       # 存储所有对应的 SNR 值

print("开始提取原始 I/Q 数据、标签和 SNR...")
# 遍历 radio_data 字典中的每一个 (调制类型, SNR) 键
for key in radio_data.keys():
    mod, snr = key          # 解包元组键
    signals = radio_data[key] # 获取该键对应的 I/Q 数据块 (N, 2, 128)
    all_signals_iq.append(signals)

    num_samples = signals.shape[0] # 获取当前数据块中的样本数量
    all_labels_str.extend([mod] * num_samples) # 为每个样本添加调制类型标签
    all_snrs_list.extend([snr] * num_samples)  # 为每个样本添加 SNR 值

# 将列表堆叠成 NumPy 数组
# np.vstack 用于垂直堆叠数组，适合将多个 (N_i, 2, 128) 的数据块合并为 (sum(N_i), 2, 128)
X_iq = np.vstack(all_signals_iq)
y_labels = np.array(all_labels_str) # 转换为 NumPy 数组
snr_array = np.array(all_snrs_list)   # 转换为 NumPy 数组

print(f"原始 I/Q 数据 X_iq 形状: {X_iq.shape}")
print(f"原始标签 y_labels 形状: {y_labels.shape}")
print(f"SNR 数组 snr_array 形状: {snr_array.shape}")

开始提取原始 I/Q 数据、标签和 SNR...
原始 I/Q 数据 X_iq 形状: (220000, 2, 128)
原始标签 y_labels 形状: (220000,)
SNR 数组 snr_array 形状: (220000,)


In [ ]:
# ## 5. 特征提取 (统计、频谱、HOC 特征)
# 从原始 I/Q 数据中提取适用于传统机器学习模型的特征。
# 这些特征包括信号的统计特性、频域特性和高阶累积量。

def extract_features_vectorized(iq_data):
    """
    从 I/Q 数据中提取多种特征（统计、频谱、HOC）的向量化版本。
    输入: iq_data (numpy array): 形状为 (N, 2, L) 的 I/Q 数据，N是样本数，L是信号长度。
    输出: numpy array: 形状为 (N, num_features) 的特征矩阵。
    """
    N, _, L = iq_data.shape # N: 样本数量, L: 每个信号的采样点数 (128)
    num_features = 16       # 定义提取的特征总数
    features = np.zeros((N, num_features), dtype=np.float32) # 初始化特征矩阵
    epsilon = 1e-10         # 一个小常数，用于防止计算中出现除以零或log(0)的错误

    print(f"开始向量化提取 {num_features} 个特征...")
    start_time = time.time()

    # 将 I/Q 数据合并为复数信号: s = I + jQ
    # iq_data[:, 0, :] 是所有样本的 I 分量
    # iq_data[:, 1, :] 是所有样本的 Q 分量
    s_all = iq_data[:, 0, :] + 1j * iq_data[:, 1, :]

    # --- 幅度与相位统计特征 ---
    amp_all = np.abs(s_all)  # 计算所有样本的瞬时幅度
    # 计算瞬时相位，np.unwrap 用于校正相位跳变 (从 -pi 到 pi)
    phase_all = np.unwrap(np.arctan2(s_all.imag, s_all.real), axis=1)
    # 计算相邻相位点之间的差分，prepend 用于在开头补齐，保持维度一致
    phase_diff_all = np.diff(phase_all, axis=1, prepend=phase_all[:, [0]])

    features[:, 0] = np.mean(amp_all, axis=1)           # 幅度均值
    features[:, 1] = np.std(amp_all, axis=1)            # 幅度标准差
    features[:, 2] = skew(amp_all, axis=1)              # 幅度偏度
    features[:, 3] = kurtosis(amp_all, axis=1)          # 幅度峰度
    features[:, 4] = np.mean(phase_diff_all, axis=1)    # 相位差分均值
    features[:, 5] = np.std(phase_diff_all, axis=1)     # 相位差分标准差
    features[:, 6] = skew(phase_diff_all, axis=1)       # 相位差分偏度
    features[:, 7] = kurtosis(phase_diff_all, axis=1)   # 相位差分峰度

    # --- 频谱特征 ---
    freqs = fftfreq(L)                     # 计算FFT对应的频率点
    freqs_shifted = fftshift(freqs)        # 将零频率移到频谱中心
    fft_s_all = fft(s_all, axis=1)         # 对每个样本信号进行FFT
    psd_all = np.abs(fft_s_all)**2 / L     # 计算功率谱密度 (PSD)
    psd_shifted_all = fftshift(psd_all, axes=1) # 将PSD的零频率移到中心

    features[:, 8] = np.max(psd_shifted_all, axis=1) # 最大功率谱密度值
    argmax_indices = np.argmax(psd_shifted_all, axis=1) # 最大PSD值对应的索引
    features[:, 9] = freqs_shifted[argmax_indices]     # 最大PSD值对应的频率

    # 谱平坦度: 功率谱的几何平均数 / 算术平均数。值越接近1，频谱越平坦。
    gmean_psd = gmean(psd_shifted_all + epsilon, axis=1) # 加epsilon防止log(0)
    mean_psd = np.mean(psd_shifted_all, axis=1)
    features[:, 10] = gmean_psd / (mean_psd + epsilon) # 加epsilon防止除0

    # 谱质心: 功率谱的加权平均频率，反映频谱能量主要集中的区域。
    spectral_centroid_num = np.sum(freqs_shifted * psd_shifted_all, axis=1)
    spectral_centroid_den = np.sum(psd_shifted_all, axis=1)
    features[:, 11] = spectral_centroid_num / (spectral_centroid_den + epsilon) # 加epsilon防止除0

    # --- 高阶累积量 (HOC) 特征 ---
    # HOC 对信号的非高斯性和对称性敏感，有助于区分调制类型
    s_mean = np.mean(s_all, axis=1, keepdims=True) # 计算每个信号的均值，用于中心化
    s_centered_all = s_all - s_mean                # 中心化信号 (减去均值)
    s_conj_all = np.conj(s_centered_all)           # 中心化信号的共轭

    # 二阶累积量
    C20_all = np.mean(s_centered_all**2, axis=1)              # C20 = E[s_c(t)^2]
    C21_all = np.mean(s_centered_all * s_conj_all, axis=1)    # C21 = E[s_c(t)s_c*(t)] = E[|s_c(t)|^2] (信号功率)
    features[:, 12] = np.abs(C20_all)
    features[:, 13] = np.abs(C21_all)

    # 四阶累积量 (简化形式，基于零均值假设)
    # C40 = E[s_c^4] - 3 * (E[s_c^2])^2 = E[s_c^4] - 3 * C20^2
    C40_all = np.mean(s_centered_all**4, axis=1) - 3 * (C20_all**2)
    # C42 = E[|s_c|^4] - |E[s_c^2]|^2 - 2 * (E[|s_c|^2])^2
    #     = E[(s_c s_c*)^2] - |C20|^2 - 2 * C21^2
    C42_all = np.mean((s_centered_all * s_conj_all)**2, axis=1) - (np.abs(C20_all)**2) - 2 * (C21_all**2)
    features[:, 14] = np.abs(C40_all)
    features[:, 15] = np.abs(C42_all)

    # --- 处理 NaN 或 Inf ---
    # 检查特征矩阵中是否存在 NaN (Not a Number) 或 Inf (Infinity)
    if np.isnan(features).any() or np.isinf(features).any():
        print("警告: 特征矩阵中检测到 NaN 或 Inf 值，将使用 np.nan_to_num 进行替换。")
        # np.nan_to_num 将 NaN 替换为0.0，正无穷大替换为大的正数，负无穷大替换为小的负数 (或指定的常数)
        features = np.nan_to_num(features, nan=0.0, posinf=0.0, neginf=0.0)

    end_time = time.time()
    print(f"向量化特征提取完成。耗时: {end_time - start_time:.2f} 秒")
    return features

# 调用特征提取函数
X_features = extract_features_vectorized(X_iq)
print(f"特征矩阵 X_features 形状: {X_features.shape}")

开始向量化提取 16 个特征...
向量化特征提取完成。耗时: 4.06 秒
特征矩阵 X_features 形状: (220000, 16)


In [6]:
# ## 6. 标签编码 (整数编码)
# 将字符串形式的调制类型标签（如 'QPSK', '8PSK'）转换为机器学习模型可接受的整数格式 (如 0, 1, 2...)。

label_encoder = LabelEncoder() # 初始化标签编码器
# fit_transform 方法会学习标签与整数的映射关系，并转换原始标签
y_numerical = label_encoder.fit_transform(y_labels)

print(f"数值标签形状: {y_numerical.shape}")
print(f"前 10 个数值标签: {y_numerical[:10]}") # 显示转换后的前10个标签以供检查

# 保存标签映射关系 (整数 -> 原始字符串标签)
# label_encoder.classes_ 存储了所有不重复的原始类别标签，其顺序与编码后的整数对应
label_mapping = {i: label for i, label in enumerate(label_encoder.classes_)}
print(f"\n标签映射关系: {label_mapping}")
num_classes = len(label_mapping) # 获取类别总数
print(f"类别总数: {num_classes}")

数值标签形状: (220000,)
前 10 个数值标签: [9 9 9 9 9 9 9 9 9 9]

标签映射关系: {0: np.str_('8PSK'), 1: np.str_('AM-DSB'), 2: np.str_('AM-SSB'), 3: np.str_('BPSK'), 4: np.str_('CPFSK'), 5: np.str_('GFSK'), 6: np.str_('PAM4'), 7: np.str_('QAM16'), 8: np.str_('QAM64'), 9: np.str_('QPSK'), 10: np.str_('WBFM')}
类别总数: 11


In [ ]:
# ## 7. 数据集划分
#
# 将特征矩阵 (X_features) 和整数标签 (y_numerical) 以及对应的 SNR 值
# 划分为训练集、验证集和测试集。
# 使用分层抽样 (stratify) 确保在划分后的各个子集中，类别（调制类型）的分布比例与原始数据集大致相同。

# 定义划分比例
test_size = 0.2  # 20% 的数据作为测试集
# val_size_relative 是相对于 (训练集+验证集) 的比例。
# 最终比例：训练集 60%，验证集 20%，测试集 20%。
val_size_relative = 0.25

# 第一次划分：分离出测试集
# X_features: 特征数据
# y_numerical: 整数标签
# snr_array: 每个样本对应的SNR值
# test_size: 测试集比例
# random_state: 随机种子，确保每次划分结果一致，便于复现
# stratify=y_numerical: 基于标签y_numerical进行分层抽样
X_train_val_feat, X_test_feat, y_train_val_num, y_test_num, snr_train_val, snr_test = train_test_split(
    X_features, y_numerical, snr_array,
    test_size=test_size,
    random_state=42,
    stratify=y_numerical # 确保测试集中各类别比例与整体一致
)

# 第二次划分：从剩余的训练验证集中分离出验证集
X_train_feat, X_val_feat, y_train_num, y_val_num, snr_train, snr_val = train_test_split(
    X_train_val_feat, y_train_val_num, snr_train_val,
    test_size=val_size_relative, # 注意此处的 test_size 是相对于 X_train_val_feat 的
    random_state=42,
    stratify=y_train_val_num # 确保验证集中各类别比例与 X_train_val_feat 一致
)

print("数据集划分完成。")
print(f"训练集:   X_feat={X_train_feat.shape}, y_num={y_train_num.shape}, SNR={snr_train.shape}")
print(f"验证集:   X_feat={X_val_feat.shape}, y_num={y_val_num.shape}, SNR={snr_val.shape}")
print(f"测试集:   X_feat={X_test_feat.shape}, y_num={y_test_num.shape}, SNR={snr_test.shape}")

数据集划分完成。
训练集:   X_feat=(132000, 16), y_num=(132000,), SNR=(132000,)
验证集:   X_feat=(44000, 16), y_num=(44000,), SNR=(44000,)
测试集:   X_feat=(44000, 16), y_num=(44000,), SNR=(44000,)


In [ ]:
# ## 8. 特征缩放 (标准化)
#
# 对特征进行标准化（Z-score scaling），使其均值为 0，标准差为 1。
# 标准化公式: z = (x - u) / s，其中 u 是均值，s 是标准差。
# - 这对于 SVM 这类依赖距离计算的算法至关重要。
# - 对于 RF 这类基于树的模型，虽然不是严格必需，但通常无害，有时也能帮助改善性能或收敛。
#
# 重要: `StandardScaler` 必须在训练集上 `fit` (计算均值和标准差)，
# 然后用相同的 scaler (相同的均值和标准差) 对训练集、验证集和测试集进行 `transform`。
# 这是为了防止测试集或验证集的信息泄露到训练过程中。

scaler = StandardScaler() # 初始化标准化缩放器

print("正在进行特征缩放 (标准化)...")

# 在训练集上拟合 scaler (计算训练集的均值和标准差) 并转换训练集
X_train_scaled = scaler.fit_transform(X_train_feat)

# 使用在训练集上拟合得到的 scaler (即使用训练集的均值和标准差) 来转换验证集和测试集
X_val_scaled = scaler.transform(X_val_feat)
X_test_scaled = scaler.transform(X_test_feat)

print("特征缩放完成。")
print(f"缩放后训练集特征形状: {X_train_scaled.shape}")
# 验证缩放效果：缩放后的训练集特征均值应接近0，标准差应接近1
print(f"训练集缩放后均值 (应接近0): {np.mean(X_train_scaled, axis=0)}")
print(f"训练集缩放后标准差 (应接近1): {np.std(X_train_scaled, axis=0)}")

正在进行特征缩放 (标准化)...
特征缩放完成。
缩放后训练集特征形状: (132000, 16)
训练集缩放后均值 (应接近0): [-1.4610490e-06 -1.7494866e-08 -2.1965667e-09 -2.3342331e-08
 -2.0671072e-08  9.2251735e-09 -8.4207361e-09  4.9496691e-09
  1.0049704e-08 -3.4702052e-07 -4.5254374e-09  5.3677343e-09
 -1.0132338e-08  6.7461619e-09 -7.2820335e-08  1.2791931e-07]
训练集缩放后标准差 (应接近1): [1.0004851  0.99999124 0.99999046 0.999986   0.9999789  0.99999976
 0.9999838  0.99999464 0.9999907  1.0000594  0.9999849  0.9999608
 0.9999801  0.9999851  0.9998059  0.99980336]


In [ ]:
# ## 9. 保存处理后的数据
#
# 将处理好的缩放后特征矩阵、整数标签、SNR 值、标签映射关系和缩放器对象保存到文件，
# 以便后续的模型训练脚本可以方便地加载和使用这些预处理成果。

# 创建保存目录 (如果目录不存在)
output_dir = 'processed_ml_features'
os.makedirs(output_dir, exist_ok=True) # exist_ok=True 表示如果目录已存在则不报错

print(f"\n正在将处理后的数据保存到 '{output_dir}' 目录...")

# 保存缩放后的特征数据 (NumPy 数组以 .npy 格式保存)
np.save(os.path.join(output_dir, 'X_train_scaled.npy'), X_train_scaled)
np.save(os.path.join(output_dir, 'X_val_scaled.npy'), X_val_scaled)
np.save(os.path.join(output_dir, 'X_test_scaled.npy'), X_test_scaled)

# 保存整数标签
np.save(os.path.join(output_dir, 'y_train.npy'), y_train_num)
np.save(os.path.join(output_dir, 'y_val.npy'), y_val_num)
np.save(os.path.join(output_dir, 'y_test.npy'), y_test_num)

# 保存SNR值 (用于后续按SNR分析模型性能)
np.save(os.path.join(output_dir, 'snr_train.npy'), snr_train)
np.save(os.path.join(output_dir, 'snr_val.npy'), snr_val)
np.save(os.path.join(output_dir, 'snr_test.npy'), snr_test)

# 保存标签映射关系
np.save(os.path.join(output_dir, 'label_mapping.npy'), label_mapping)

# 保存缩放器 scaler 对象 (使用 pickle 进行序列化)
# scaler 对象包含了训练集的均值和标准差，在对新数据进行预测时需要用它来做同样的特征缩放
scaler_path = os.path.join(output_dir, 'scaler.pkl')
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f"缩放器已保存到: {scaler_path}")

print("\n数据保存完成。")
print(f"文件列表 ({output_dir}):")
for filename in os.listdir(output_dir):
    print(f" - {filename}")


正在将处理后的数据保存到 'processed_ml_features' 目录...
缩放器已保存到: processed_ml_features/scaler.pkl

数据保存完成。
文件列表 (processed_ml_features):
 - X_train_scaled.npy
 - snr_val.npy
 - y_val.npy
 - snr_train.npy
 - scaler.pkl
 - label_mapping.npy
 - y_train.npy
 - y_test.npy
 - X_test_scaled.npy
 - snr_test.npy
 - X_val_scaled.npy


In [ ]:
# ## 10. 总结
#
# 此 Notebook 为 SVM, RF 预处理了 RadioML 2016.10a 数据集：
# 1. 加载了原始数据。
# 2. 提取了 I/Q 信号、标签和 SNR。
# 3. 提取了统计、频谱、HOC 特征，生成特征矩阵。
# 4. 进行了标签整数编码。
# 5. 划分了训练/验证/测试集 (60/20/20 比例，分层抽样)。
# 6. 对特征数据进行了标准化。
# 7. 将处理后的数据和辅助文件 (scaler, label_mapping) 保存到了 `processed_ml_features` 目录。
#
# 这些数据可用于后续的模型训练和评估。
#
# --- End of Notebook ---